In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import os


In [2]:
page = 1 

states_dict = {
    1: 'Andaman and Nicobar Islands',
    2: 'Andhra Pradesh',
    3: 'Arunachal Pradesh',
    4: 'Assam',
    5: 'Bihar',
    6: 'Chandigarh',
    7: 'Chhattisgarh',
    8: 'Dadra and Nagar Haveli and Daman and Diu',
    9: 'Delhi',
    10: 'Goa',
    11: 'Gujarat',
    12: 'Haryana',
    13: 'Himachal Pradesh',
    14: 'Jammu and Kashmir',
    15: 'Jharkhand',
    16: 'Karnataka',
    17: 'Kerala',
    18: 'Ladakh',
    19: 'Lakshadweep',
    20: 'Madhya Pradesh',
    21: 'Maharashtra',
    22: 'Manipur',
    23: 'Meghalaya',
    24: 'Mizoram',
    25: 'Nagaland',
    26: 'Odisha',
    27: 'Puducherry',
    28: 'Punjab',
    29: 'Rajasthan',
    30: 'Sikkim',
    31: 'Tamil Nadu',
    32: 'Telangana',
    33: 'Tripura',
    34: 'Uttar Pradesh',
    35: 'Uttarakhand',
    36: 'West Bengal'
}


## 2024 Data:

In [4]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

for state in range(1, 37):
    page = 1
    while True:
        url = f"https://myneta.info/LokSabha2024/index.php?action=show_constituencies&state_id={state}&page={page}"
        driver.get(url)

        # Wait until the table is loaded
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'w3-responsive')))
        except:
            break

        table_elements = driver.find_elements(By.CLASS_NAME, 'w3-responsive')

        for table in table_elements:
            html_content = table.get_attribute("outerHTML")
            
            # Use BeautifulSoup to parse the HTML and remove <script> tags
            soup = BeautifulSoup(html_content, 'html.parser')
            for script in soup(["script", "style"]):
                script.extract()  # Remove the script and style tags
            
            cleaned_html = soup.prettify()

            with open(f"../data/html/candidate-background-2024/{states_dict[state]}_{page}.html", "w", encoding="utf-8") as f:
                f.write(cleaned_html)

        # Check if there is a next page
        try:
            pagination_div = driver.find_element(By.XPATH, "//div[center]/center")
            pagination_soup = BeautifulSoup(pagination_div.get_attribute("outerHTML"), 'html.parser')
            pages_text = pagination_soup.find("small").text.strip()
            total_pages = int(pages_text.split()[-2])

            if page < total_pages:
                page += 1
            else:
                break
        except:
            break

driver.close()

## 2019 Data:

In [7]:
# State dictionary for the 2019 election (with their unique IDs)
states_dict = {
    34: 'Andhra Pradesh',
    35: 'Arunachal Pradesh',
    36: 'Assam',
    37: 'Bihar',
    38: 'Goa',
    39: 'Gujarat',
    40: 'Haryana',
    41: 'Himachal Pradesh',
    42: 'Jammu and Kashmir',
    43: 'Karnataka',
    44: 'Kerala',
    45: 'Madhya Pradesh',
    46: 'Maharashtra',
    47: 'Manipur',
    48: 'Meghalaya',
    49: 'Mizoram',
    50: 'Nagaland',
    51: 'Odisha',
    52: 'Punjab',
    53: 'Rajasthan',
    54: 'Sikkim',
    55: 'Tamil Nadu',
    56: 'Telangana',
    57: 'Tripura',
    58: 'Uttar Pradesh',
    59: 'Chhattisgarh',  # Adjusted index
    60: 'Jharkhand',      # Adjusted index
    61: 'Uttarakhand'     # Adjusted index
}

# Union territory dictionary for the 2019 election (with their unique IDs)
union_territories_dict = {
    62: 'Andaman and Nicobar Islands',
    63: 'Chandigarh',
    64: 'Dadra and Nagar Haveli',
    65: 'Daman and Diu',
    66: 'Lakshadweep',
    67: 'National Capital of Delhi',
    68: 'Puducherry'
}

# Initialize WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

# Function to fetch data for a given state or union territory ID
def fetch_data_for_id(state_id, state_name):
    page = 1
    while True:
        url = f"https://myneta.info/LokSabha2019/index.php?action=show_constituencies&state_id={state_id}&page={page}"
        driver.get(url)

        # Wait until the table is loaded
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'w3-responsive')))
        except:
            break

        table_elements = driver.find_elements(By.CLASS_NAME, 'w3-responsive')

        for table in table_elements:
            html_content = table.get_attribute("outerHTML")
            
            # Use BeautifulSoup to parse the HTML and remove <script> tags
            soup = BeautifulSoup(html_content, 'html.parser')
            for script in soup(["script", "style"]):
                script.extract()  # Remove the script and style tags
            
            cleaned_html = soup.prettify()

            # Save HTML content to file
            with open(f"../data/html/candidate-background-2019/{state_name}_{page}.html", "w", encoding="utf-8") as f:
                f.write(cleaned_html)

        # Check if there is a next page
        try:
            pagination_div = driver.find_element(By.XPATH, "//div[center]/center")
            pagination_soup = BeautifulSoup(pagination_div.get_attribute("outerHTML"), 'html.parser')
            pages_text = pagination_soup.find("small").text.strip()
            total_pages = int(pages_text.split()[-2])

            if page < total_pages:
                page += 1
            else:
                break
        except:
            break

# Fetch data for states
for state_id, state_name in states_dict.items():
    fetch_data_for_id(state_id, state_name)

# Fetch data for union territories
for ut_id, ut_name in union_territories_dict.items():
    fetch_data_for_id(ut_id, ut_name)

# Close the driver after fetching all data
driver.close()


In [7]:
link = 'https://www.indiavotes.com/state/summary/1'
driver = webdriver.Chrome()
driver.get(link)

# Wait until the table is loaded
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'CensusData')))

table_elements = driver.find_elements(By.CLASS_NAME, 'CensusData')

for table in table_elements:
    html_content = table.get_attribute("outerHTML")
    
    # Use BeautifulSoup to parse the HTML and remove <script> tags
    soup = BeautifulSoup(html_content, 'html.parser')
    # for script in soup(["script", "style"]):
    #     script.extract()  # Remove the script and style tags
    
    cleaned_html = soup.prettify()
    print(cleaned_html)

    # Save HTML content to file
    with open(f"../data/html/state-census/Assam.html", "w", encoding="utf-8") as f:
        f.write(cleaned_html)

## Census Data:

In [9]:
page = 1 

states_dict = {
    34: 'Andaman and Nicobar Islands',
    62: 'Andhra Pradesh',
    41: 'Arunachal Pradesh',
    1: 'Assam',
    58: 'Bihar',
    35: 'Chandigarh',
    54: 'Chhattisgarh',
    36: 'Dadra and Nagar Haveli and Daman and Diu',
    52: 'Daman and Diu',
    57: 'Delhi',
    51: 'Goa',
    29: 'Gujarat',
    31: 'Haryana',
    22: 'Himachal Pradesh',
    32: 'Jammu and Kashmir',
    53: 'Jharkhand',
    43: 'Karnataka',
    28: 'Kerala',
    48: 'Lakshadweep',
    59: 'Madhya Pradesh',
    30: 'Maharashtra',
    24: 'Manipur',
    44: 'Meghalaya',
    45: 'Mizoram',
    33: 'Nagaland',
    6: 'Odisha',
    39: 'Puducherry',
    7: 'Punjab',
    14: 'Rajasthan',
    46: 'Sikkim',
    40: 'Tamil Nadu',
    61: 'Telangana',
    25: 'Tripura',
    60: 'Uttar Pradesh',
    56: 'Uttarakhand',
    9: 'West Bengal'
}

In [20]:
def extract_and_save_table(driver, url, element_id, file_prefix):
    """
    Extracts the table with the specified ID from the webpage and saves it as an HTML file.
    
    Args:
    - driver: Selenium WebDriver object
    - url: URL of the webpage
    - element_id: The ID of the HTML element (table) to extract
    - file_prefix: Prefix for the output HTML file name
    """
    driver.get(url)
    try:
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, 'CensusData')))
        
        # Check if the specified element exists
        if driver.find_elements(By.ID, element_id):
            elements = driver.find_elements(By.ID, element_id)
            for data in elements:
                html_content = data.get_attribute("outerHTML")
                soup = BeautifulSoup(html_content, 'html.parser')
                cleaned_html = soup.prettify()

                # Save to an HTML file
                with open(f"../data/html/state-census/{file_prefix}_{element_id}.html", "w", encoding="utf-8") as f:
                    f.write(cleaned_html)

            print(f"Data for {file_prefix} ({element_id}) saved successfully.")
        else:
            print(f"{element_id} not found for {file_prefix}. Skipping this table.")
            return False  # Return False if the element is not found

    except Exception as e:
        print(f"Error fetching {element_id} for {file_prefix}: {e}")
        return False  # Return False if an error occurs

    return True  # Return True if the table is successfully saved

# Set up the Selenium WebDriver
driver = webdriver.Chrome()

# Loop through each state in the dictionary
for index, state_name in states_dict.items():
    state_url = f"https://www.indiavotes.com/state/summary/{index}"

    # Extract and save CensusData table
    census_saved = extract_and_save_table(driver, state_url, 'CensusData', state_name)

    # Only attempt to extract the age_dist table if CensusData was successfully saved
    if census_saved:
        age_dist_saved = extract_and_save_table(driver, state_url, 'age_dist', state_name)

        # If age_dist is not found, skip to the next state
        if not age_dist_saved:
            continue

# Close the browser after processing all states
driver.close()

Data for Andaman and Nicobar Islands (CensusData) saved successfully.
Data for Andaman and Nicobar Islands (age_dist) saved successfully.
Error fetching CensusData for Andhra Pradesh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF73DA4B632+29090]
	(No symbol) [0x00007FF73D9BE6E9]
	(No symbol) [0x00007FF73D87B1CA]
	(No symbol) [0x00007FF73D8CEFD7]
	(No symbol) [0x00007FF73D8CF22C]
	(No symbol) [0x00007FF73D9197F7]
	(No symbol) [0x00007FF73D8F672F]
	(No symbol) [0x00007FF73D9165D9]
	(No symbol) [0x00007FF73D8F6493]
	(No symbol) [0x00007FF73D8C09B1]
	(No symbol) [0x00007FF73D8C1B11]
	GetHandleVerifier [0x00007FF73DD6883D+3294125]
	GetHandleVerifier [0x00007FF73DDB4423+3604371]
	GetHandleVerifier [0x00007FF73DDAA2E7+3563095]
	GetHandleVerifier [0x00007FF73DB06F16+797318]
	(No symbol) [0x00007FF73D9C986F]
	(No symbol) [0x00007FF73D9C5454]
	(No symbol) [0x00007FF73D9C55E0]
	(No symbol) [0x00007FF73D9B4A7F]
	BaseThreadInitThunk [0x00007FFB4E43257D+29]
	RtlUserThreadStart [0x00007FFB4EE4A